## 08- Intermediate data

Exploration of intermediate results generated with Claudia's code to avoid using simulator. :) 

Enjoy! 

In [1]:
import os
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
from Utils import *

In [2]:
rootdir = '/'.join(os.getcwd().split('/')[:-1]) #eliminate folder Python from the path
data_path = rootdir + "/Project_EADARP/Dynamic_eADARP/mysols3"

df_nodes = pd.read_pickle('./tables/nodes.pkl')

In [3]:
#m = np.mean(df_nodes['Lat'])
#s = np.std(df_nodes['Lat'])

#m_lon = np.mean(df_nodes['Lon'])
#s_lon = np.std(df_nodes['Lon'])

#df_nodes['rel_lat'] = (df_nodes['Lat'] - m)/s
#df_nodes['rel_lon'] = (df_nodes['Lon'] - m_lon)/s_lon

In [7]:
min_lat = np.min(df_nodes['Lat'])
max_lat = np.max(df_nodes['Lat'])

min_lon = np.min(df_nodes['Lon'])
max_lon = np.max(df_nodes['Lon'])

df_nodes['rel_lat'] = (df_nodes['Lat'] - min_lat)/(max_lat - min_lat)
df_nodes['rel_lon'] = (df_nodes['Lon'] - min_lon)/(max_lon - min_lon)

In [9]:
print(min_lat)
print(max_lat)

print(min_lon)
print(max_lon)

37.772949
37.790377
-122.424296
-122.405883


In [4]:
df_nodes

,ID,Lat,Lon,rel_lat,rel_lon
0,1,37.772949,-122.422210,0.000000,0.113290
1,2,37.773070,-122.422046,0.006943,0.122196
2,3,37.774035,-122.420740,0.062314,0.193124
3,4,37.773170,-122.422224,0.012681,0.112529
4,5,37.773615,-122.422330,0.038214,0.106772
...,...,...,...,...,...
252,253,37.790331,-122.407513,0.997361,0.911476
253,254,37.790366,-122.407241,0.999369,0.926248
254,255,37.773022,-122.421949,0.004189,0.127464
255,256,37.785791,-122.406264,0.736860,0.979308


In [5]:
def parse_int_solution(file):
    r_file = open(file)
    line = r_file.readline()
    client = 0
    vehicle = 0
    
    status = []
    
    while line:
        if len(line.rstrip().split()) > 0:
            if line.rstrip().split()[0] == '-----CLIENT':
                client = int(line.rstrip().split()[1]) + 1
                # read two lines because there is a black in the middle
                line = r_file.readline()
                while len(line.rstrip().split()) == 0:
                    line = r_file.readline()
            
            elif line.rstrip().split()[0] == '###############################':
                vehicle = int(line.rstrip().split()[2]) - 1
                line = r_file.readline()
                current_node = -1
                current_passengers = 0
                queue_passengers = 0
                future_node = -1
                
                while len(line.rstrip().split()) > 0 and line.rstrip().split()[0] != '###############################' \
                and line.rstrip().split()[0] != 'Best_vehicle':
                    
                    node,wait,service_start, service_end,arr,dep,service_t,load,battery,charging,abottom,btop =\
                    line.rstrip().split()[0].split(',')
                    node = int(node)
    
                    if current_node == -1:
                        current_node = min(node, 201)
                        current_passengers = int(load)
                    if future_node == -1 and node != current_node: 
                        future_node = min(node, 201)
                    if node in range(100) and node != current_node:
                        queue_passengers += 1
                    
                    line = r_file.readline()
                
                my_veh = {'client': client, 'vehicle':vehicle, 'current_node': current_node, 'load': current_passengers,\
                'queue': queue_passengers, 'future_node': future_node}
                status.append(my_veh)
            
            elif line.rstrip().split()[0] == 'Best_vehicle':
                status.append({'vehicle_sol': line.rstrip().split()[1]})
                line = r_file.readline()
            else:
                line = r_file.readline()
        
        else:
            line = r_file.readline()
    
    r_file.close()
            
    return status
        
def init_client(n_vehicles=30):
    # initialize vehicles
    status = []
    for i in range(n_vehicles):
        my_veh = {'client': 1, 'vehicle':i, 'current_node': 201, 'load': 0,\
                'queue': 0, 'future_node': 201}
        status.append(my_veh)
    return status

In [6]:
dfs = []
sols = []
for inst in range(2,100): #2,21
    ins = []
    sol = []
    for c in range(1,101): #1,101
        try:
            file = 'solution_instance_{}_at_{}.txt'.format(inst, c)
            file = data_path +'/'+ file
            ins = ins + parse_int_solution(file)
            
            sol.append({'client': int(c), 'vehicle':float(ins[-1]['vehicle_sol'])})
            ins = ins[:-1]
            
        except:
            print('not found client')
        
    ins = ins[:-1] + init_client()
    df = pd.DataFrame(ins)
    dfs.append(df)
    sols.append(sol)

In [7]:
def assign_location(df):
    df_loc = df.merge(df_nodes, left_on='current_node', right_on='ID', how='left').\
    drop(['ID', 'rel_lat', 'rel_lon', 'current_node'], axis=1)
    
    df_loc.rename(columns={'Lat':'current_lat', 'Lon':'current_lon'}, inplace=True)
    
    df_loc = df_loc.merge(df_nodes, left_on='future_node', right_on='ID', how='left').\
    drop(['ID', 'rel_lat', 'rel_lon', 'future_node'], axis=1)
    
    df_loc.rename(columns={'Lat':'future_lat', 'Lon':'future_lon'}, inplace=True)
    
    return df_loc
                  
        

In [15]:
def assign_location_rel(df):
    df_loc = df.merge(df_nodes, left_on='current_node', right_on='ID', how='left').\
    drop(['ID', 'Lat', 'Lon', 'current_node'], axis=1)
    
    df_loc.rename(columns={'rel_lat':'current_lat', 'rel_lon':'current_lon'}, inplace=True)
    
    df_loc = df_loc.merge(df_nodes, left_on='future_node', right_on='ID', how='left').\
    drop(['ID', 'Lat', 'Lon', 'future_node'], axis=1)
    
    df_loc.rename(columns={'rel_lat':'future_lat', 'rel_lon':'future_lon'}, inplace=True)
    
    df_loc.current_lat = round(df_loc.current_lat, 3)
    df_loc.current_lon = round(df_loc.current_lon, 3)
    df_loc.future_lat = round(df_loc.future_lat, 3)
    df_loc.future_lon = round(df_loc.future_lon, 3)
    
    return df_loc

In [ ]:
dfs_loc = []
for df in dfs:
    dfs_loc.append(assign_location(df))

In [18]:
dfs_loc_rel = []
for df in dfs:
    dfs_loc_rel.append(assign_location_rel(df))

In [21]:
"""
convert to list of dict{'client': array[client, state]}
"""

for TABLE, df in enumerate(dfs_loc_rel):
    case = {}
    sol = {}
    
    print('\rProcessing TABLE {}'.format(TABLE), end=" ")
    for c in list(set(df.client))[:-1]:
        X = []
        client_data = [float(df_nodes[df_nodes.ID == c].rel_lat), float(df_nodes[df_nodes.ID == c].rel_lon), \
                      float(df_nodes[df_nodes.ID == c + 100].rel_lat), float(df_nodes[df_nodes.ID == c + 100].rel_lon)]
        state = df[df.client == c]
        X.append(client_data)
        
        for v in list(set(df.vehicle)):
            df_v = state[state.vehicle == v]
            try:
                X.append([int(df_v.load), int(df_v.queue), float(df_v.current_lat), \
                      float(df_v.current_lon), float(df_v.future_lat), float(df_v.future_lon)])
            except:
                print(df_v)
                
        solution = sols[TABLE][c-1]
        assert solution['client'] == c
        
        if not math.isnan(solution['vehicle']):
            sol[c] = solution['vehicle'] - 1
            case[c] = X
    
    np.save('./minmax_data/data_vector_{}'.format(TABLE+1), case)
    np.save('./minmax_data/data_vector_y_{}'.format(TABLE+1), sol)
        

Processing TABLE 97 

In [3]:
data = np.load('./minmax_data/data_vector_{}.npy'.format(2), allow_pickle=True).tolist()

In [4]:
data

{1: [[0.0, 0.11328952370563729, 0.5429768189179438, 0.1890512138167989],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 0.82, 0.69, 0.82, 0.69],
  [0, 0, 

In [5]:
data = np.load('./minmax_data/data_vector_y_{}.npy'.format(2), allow_pickle=True).tolist()

In [6]:
data

{1: 0.0,
 2: 1.0,
 3: 2.0,
 4: 2.0,
 5: 3.0,
 6: 4.0,
 7: 5.0,
 8: 5.0,
 9: 3.0,
 10: 5.0,
 11: 6.0,
 12: 7.0,
 13: 1.0,
 14: 3.0,
 15: 1.0,
 16: 1.0,
 17: 6.0,
 18: 6.0,
 19: 0.0,
 20: 5.0,
 21: 8.0,
 22: 0.0,
 23: 0.0,
 24: 8.0,
 25: 9.0,
 26: 10.0,
 27: 11.0,
 28: 11.0,
 29: 7.0,
 30: 3.0,
 31: 11.0,
 32: 12.0,
 33: 12.0,
 34: 13.0,
 35: 0.0,
 36: 14.0,
 37: 7.0,
 38: 15.0,
 39: 16.0,
 40: 14.0,
 41: 10.0,
 42: 17.0,
 43: 0.0,
 44: 1.0,
 45: 15.0,
 46: 7.0,
 47: 10.0,
 48: 16.0,
 49: 14.0,
 50: 12.0,
 51: 18.0,
 52: 12.0,
 53: 15.0,
 54: 18.0,
 55: 17.0,
 56: 16.0,
 57: 0.0,
 58: 10.0,
 59: 17.0,
 60: 18.0,
 61: 14.0,
 62: 17.0,
 63: 9.0,
 64: 19.0,
 65: 20.0,
 66: 20.0,
 67: 0.0,
 68: 21.0,
 69: 20.0,
 70: 17.0,
 71: 12.0,
 72: 20.0,
 73: 7.0,
 74: 0.0,
 75: 1.0,
 76: 5.0,
 77: 20.0,
 78: 10.0,
 79: 16.0,
 80: 17.0,
 81: 13.0,
 82: 16.0,
 83: 13.0,
 84: 13.0,
 85: 22.0,
 86: 23.0,
 87: 16.0,
 88: 1.0,
 89: 1.0,
 90: 13.0,
 91: 9.0,
 92: 24.0,
 93: 24.0,
 94: 12.0,
 95: 15.0,
 96: 1